In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import duckdb

ModuleNotFoundError: No module named 'duckdb'

In [2]:
pysqldf = lambda q: pdsql.sqldf(q, globals())

# Data 04/03/2025

In [3]:
# config parameters
bss = [4096]
nprocs = [1, 32, 64]
rws = "read write rw randread randwrite randrw".split(' ')
ioengines = ["sync-sync", "sync-mmap", "async-io_uring", "async-libaio"]
asyncioDepths = [64, 128]

In [4]:
resultsDir = "data/2025-03-04-time-18-40-26-second-run-finch2/"
configPaths = {}
configNames = []
for subdirpath, subsubdirs, subfiles in os.walk(resultsDir):
    if "fio_out.txt" in subfiles:
        cpath = os.path.normpath(subdirpath)
        cname = cpath[len(resultsDir):].replace("\\", "‐")
        configNames.append(cname)
        configPaths[cname] = cpath

In [5]:
# for config in configPaths.values():
#     print(config)

In [6]:
allDataJson = {}
for cname, cpath in configPaths.items():
    with open(f"{cpath}/fio_out.txt") as datafile:
        allDataJson[cname] = json.load(datafile)

In [7]:
# quick test
# i = 77
# print(f"data for config {configNames[i]}:")
# data[configNames[i]]

# Master table generation

New idea: create one big table that has all of the data in it, and then query it using SQL

The approach I'm going with for now is to use PandaSQL

Likely should extract the below into a seperate script, and also create a script that can add columns to an existing master table (if I decide i want to profile more metrics later)

In [8]:
# master table will have columns for params as well as columns for metrics
#
# dictionary keys below are the names of these columns as they will appear in the master table
# dictionary values are the name of the corresponding index in the JSON, seperated by colons to indicate nesting
#
# param keys will be prefixed by 'c-' to specify they are part of the config
params = {
    "bSize": "job options:bs",
    "ioengine": "job options:ioengine",
    "iodepth": "job options:iodepth", # note: this will be overrided for sync ioengines to 1
    "rw": "job options:rw",
    "nproc": "job options:numjobs",
    "device": "special"
}
metrics = {
    "readBW": "read:bw",
    "writeBW": "write:bw",
    "avgreadlat": "read:lat_ns:mean",
    "avgwritelat": "write:lat_ns:mean"
} # TODO: add more metrics

paramDefaults = {
    "bSize": 4096,
    "iodepth": 1,
    "rw": "read",
    "nproc": 1
}

columns = {**{"c" + pname: ppath for pname, ppath in params.items()}, **metrics}

In [9]:
mTable = []

for cname, cjson in allDataJson.items():
    mRow = {}

    cjson = cjson["jobs"][0]
    for colname, colpath in columns.items():
        value = cjson

        if colpath == "special":
            if colname == "cdevice":
                directory = cjson["job options"]["directory"]
                if "zram" in directory:
                    value = "zram"
                elif "ssd" in directory:
                    value = "ssd"
        else:
            for key in colpath.split(':'):
                try:
                    value = value[key]
                except KeyError:
                    value = paramDefaults[key]
                
        mRow[colname] = value
        
    mTable.append(mRow)

mTable = pd.DataFrame(mTable)

In [10]:
outDir = "processed-data/2025-03-04-second-run-finch2"
mTable.to_csv(f"{outDir}/master.csv")

In [11]:
pysqldf("SELECT cbSize, crw, cioengine, ciodepth, cnproc, cdevice FROM mTable")

,cbSize,crw,cioengine,ciodepth,cnproc,cdevice
0,4096,randread,io_uring,128,1,ssd
1,4096,randread,io_uring,128,1,zram
2,4096,randread,io_uring,64,1,ssd
3,4096,randread,io_uring,64,1,zram
4,4096,randread,libaio,128,1,ssd
...,...,...,...,...,...,...
211,4096,write,libaio,64,64,zram
212,4096,write,mmap,1,64,ssd
213,4096,write,mmap,1,64,zram
214,4096,write,sync,1,64,ssd


In [ ]:
sql3